In [1]:
from dask.distributed import WorkerPlugin, get_worker, LocalCluster, Client, Variable
import time
from datetime import datetime, timedelta

In [2]:
cluster = LocalCluster(
    host="0.0.0.0",
    scheduler_kwargs={"external_address": "localhost"},
)

2025-01-20 16:00:51,974 - distributed.deploy.spec - WARNING - Cluster closed without starting up


RuntimeError: Cluster failed to start: Server.__init__() got an unexpected keyword argument 'external_address'

In [2]:
cluster = LocalCluster(
        n_workers=1,
        threads_per_worker=1,
        processes=True,
        # memory_limit="2GB",
    )
client = Client(cluster)
client.forward_logging()

{'tcp://127.0.0.1:60732': {'status': 'OK'}}

In [39]:
def dummy_task(n, var_st):
    print(f"task {n} begins...")
    st_dict = var_st.get()
    st_dict["start_time"] = datetime.now()
    var_st.set(st_dict)
    time.sleep(300)
    print(f"task {n} finished")

In [40]:
vars = []
future_dict = {}

symbol = 1
var_st = Variable()
var_st.set({"symbol":symbol})
vars.append(var_st)
future_dict[symbol] = client.submit(dummy_task, symbol, var_st)

In [41]:
symbol = 2
var_st = Variable()
var_st.set({"symbol": symbol})
vars.append(var_st)
future_dict[symbol] = client.submit(dummy_task, symbol, var_st)

In [42]:
symbol = 3
var_st = Variable()
var_st.set({"symbol": symbol})
vars.append(var_st)
future_dict[symbol] = client.submit(dummy_task, symbol, var_st)

In [79]:
for v in vars:
    try:
        vd = v.get("200ms")
        print(vd)
        if not "start_time" in vd:
            continue
        if vd["start_time"] + timedelta(seconds=60) <= datetime.now():
            ## the task is timeout. check future status. if it's still processing, cancel it.
            symbol = vd["symbol"]
            future = future_dict[symbol]
            if not future.done():
                print(f"cancelling task {symbol}")
                future.cancel()
            v.delete()
            vars.remove(v)
    except TimeoutError as e:
        print('normal timeout')
        vars.remove(v)

task 3 finished


2024-04-23 19:58:19,579 - distributed.worker - WARNING - Scheduler was unaware of this worker; shutting down.
2024-04-23 19:58:19,695 - distributed.core - INFO - Connection to tcp://127.0.0.1:60727 has been closed.


In [1]:
# Sample list of tuples
tuples_list = [
    (
        {
            "ar_layer_spec": [4, 15],
            "ar_reg": 49.16784,
            "batch_size": 300,
            "growth": "discontinuous",
            "lagged_reg_layer_spec": [256, 3],
            "n_lags": 19,
            "normalize": "soft1",
            "optimizer": "SGD",
            "seasonality_mode": "additive",
            "seasonality_reg": 47.37235,
            "topk_covar": 295,
            "trend_reg": 63.65796,
            "yearly_seasonality": 21,
        },
        1.2,
    ),
    (
        {
            "ar_layer_spec": [32, 34],
            "ar_reg": 52.31495,
            "batch_size": 400,
            "growth": "discontinuous",
            "lagged_reg_layer_spec": [512, 42],
            "n_lags": 29,
            "normalize": "soft",
            "optimizer": "SGD",
            "seasonality_mode": "multiplicative",
            "seasonality_reg": 25.50586,
            "topk_covar": 537,
            "trend_reg": 75.50941,
            "yearly_seasonality": 29,
        },
        2.3,
    ),
]

# Unpack the list of tuples and use zip to transpose
dicts, nones = zip(*tuples_list)

# Convert the resulting tuples to lists
list_dicts = list(dicts)
list_nones = list(nones)

# Output the results
print("List of dictionaries:", list_dicts)
print("List of None elements:", list_nones)

List of dictionaries: [{'ar_layer_spec': [4, 15], 'ar_reg': 49.16784, 'batch_size': 300, 'growth': 'discontinuous', 'lagged_reg_layer_spec': [256, 3], 'n_lags': 19, 'normalize': 'soft1', 'optimizer': 'SGD', 'seasonality_mode': 'additive', 'seasonality_reg': 47.37235, 'topk_covar': 295, 'trend_reg': 63.65796, 'yearly_seasonality': 21}, {'ar_layer_spec': [32, 34], 'ar_reg': 52.31495, 'batch_size': 400, 'growth': 'discontinuous', 'lagged_reg_layer_spec': [512, 42], 'n_lags': 29, 'normalize': 'soft', 'optimizer': 'SGD', 'seasonality_mode': 'multiplicative', 'seasonality_reg': 25.50586, 'topk_covar': 537, 'trend_reg': 75.50941, 'yearly_seasonality': 29}]
List of None elements: [1.2, 2.3]
